# 第三章 分组

SAC过程  
groupby函数  
聚合、过滤和变换  
apply函数  
问题与练习  



In [2]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df = pd.read_csv('./data/table.csv',index_col='ID') # 读取数据，并将ID列设为索引
df.head()
df.info()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 1101 to 2405
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   School   35 non-null     object 
 1   Class    35 non-null     object 
 2   Gender   35 non-null     object 
 3   Address  35 non-null     object 
 4   Height   35 non-null     int64  
 5   Weight   35 non-null     int64  
 6   Math     35 non-null     float64
 7   Physics  35 non-null     object 
dtypes: float64(1), int64(2), object(5)
memory usage: 2.5+ KB


## 一、 SAC （split-apply-combine）过程
### 1. 内涵
SAC指的是分组操作中的split-apply-combine过程  
其中split指基于某一些规则，将数据拆成若干组，apply是指对每一组独立地使用函数，combine指将每一组的结果组合成某一类数据结构  
### 2. apply过程
在该过程中，我们实际往往会遇到四类问题：  
 整合（Aggregation）——即分组计算统计量（如求均值、求每组元素个数）  
 变换（Transformation）——即分组对每个单元的数据进行操作（如元素标准化）  
 过滤（Filtration）——即按照某些规则筛选出一些组（如选出组内某一指标小于50的组）  
 综合问题——即前面提及的三种问题的混合  

## 二、groupby函数
经过groupby后会生成一个groupby对象，该对象本身不会返回任何东西，只有当相应的方法被调用才会起作用


**分组函数的基本内容：**  
根据某一列分组 
根据某几列分组  
组容量与组数  
组的遍历  
level参数(用于多级索引)和axis参数  

**groupby对象的特点：**  
课调用方法  
分组对象的head 和first  
分组依据  
groupby的[]操作  
连续型变量分组  

### 1. 分组函数的基本内容

In [9]:
# a 根据某一列分组
group_single = df.groupby('School')
group_single

# 经过groupby后会生成一个groupby对象，该对象本身不会返回任何东西，只有当相应的方法被调用才会起作用
# 例如取出某一个组：
group_single.get_group('S_2').head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2101,S_2,C_1,M,street_7,174,84,83.3,C
2102,S_2,C_1,F,street_6,161,61,50.6,B+
2103,S_2,C_1,M,street_4,157,61,52.5,B-
2104,S_2,C_1,F,street_5,159,97,72.2,B+
2105,S_2,C_1,M,street_4,170,81,34.2,A


In [13]:
# b 根据某几列分组
group_mul = df.groupby(['School','Class']) # groupby的时候 多列用[]框起来
group_mul.get_group(('S_1','C_1')) # 调用时 get_group用()界定

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [32]:
# c 组容量和组数
# 调用的时候最好先根据size看下里面的内容，不然在get_group的时候可能会出错
# group_mul.get_group(('S_1','C_4')) # 报错，S_1不存在C_4的类别

group_single.size()
group_mul.size()

group_single.ngroups
group_mul.ngroups

# group_single.ngroup('S_1')

School
S_1    15
S_2    20
dtype: int64

School  Class
S_1     C_1      5
        C_2      5
        C_3      5
S_2     C_1      5
        C_2      5
        C_3      5
        C_4      5
dtype: int64

2

7

In [34]:
# d 组的遍历
for name, group in group_single:
    print(name)
    display(group.head())
    
# 多列的groupby会展示所有的group组合    
for name, group in group_mul:  
    print(name)
    display(group.head())

S_1


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


S_2


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2101,S_2,C_1,M,street_7,174,84,83.3,C
2102,S_2,C_1,F,street_6,161,61,50.6,B+
2103,S_2,C_1,M,street_4,157,61,52.5,B-
2104,S_2,C_1,F,street_5,159,97,72.2,B+
2105,S_2,C_1,M,street_4,170,81,34.2,A


('S_1', 'C_1')


...

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2201,S_2,C_2,M,street_5,193,100,39.1,B
2202,S_2,C_2,F,street_7,194,77,68.5,B+
2203,S_2,C_2,M,street_4,155,91,73.8,A+
2204,S_2,C_2,M,street_1,175,74,47.2,B-
2205,S_2,C_2,F,street_7,183,76,85.4,B


('S_2', 'C_3')


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2301,S_2,C_3,F,street_4,157,78,72.3,B+
2302,S_2,C_3,M,street_5,171,88,32.7,A
2303,S_2,C_3,F,street_7,190,99,65.9,C
2304,S_2,C_3,F,street_6,164,81,95.5,A-
2305,S_2,C_3,M,street_4,187,73,48.9,B


('S_2', 'C_4')


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2401,S_2,C_4,F,street_2,192,62,45.3,A
2402,S_2,C_4,M,street_7,166,82,48.7,B
2403,S_2,C_4,F,street_6,158,60,59.7,B+
2404,S_2,C_4,F,street_2,160,84,67.7,B
2405,S_2,C_4,F,street_6,193,54,47.6,B


In [46]:
# e level参数和axis参数
# level 参数从0开始，0代表第一个索引
# level=1, axis=1 会报错
df.set_index(['Gender','School'])
df.set_index(['Gender','School']).groupby(level=1, axis=0) # 按两层索引groupvy
df.set_index(['Gender','School']).groupby(level=1, axis=0).get_group('S_1').head()

Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_1      C_1  street_1     173      63  34.0      A+
F      S_1      C_1  street_2     192      73  32.5      B+
M      S_1      C_1  street_2     186      82  87.2      B+
F      S_1      C_1  street_2     167      81  80.4      B-
       S_1      C_1  street_4     159      64  84.8      B+
M      S_1      C_2  street_5     188      68  97.0      A-
F      S_1      C_2  street_4     176      94  63.5      B-
M      S_1      C_2  street_6     160      53  58.8      A+
F      S_1      C_2  street_5     162      63  33.8       B
       S_1      C_2  street_6     167      63  68.4      B-
M      S_1      C_3  street_4     161      68  31.5      B+
F      S_1      C_3  street_1     175      57  87.7      A-
M      S_1      C_3  street_7     188      82  49.7       B
       S_1      C_3  street_2     195      70  85.2       A
F      S_1      C_3  street_5     187      69  61.7      B-
M      S_2      C_1  street_7     174      84  83.3       C
F      S_2      C_1  street_6     161      61  50.6      B+
M      S_2      C_1  street_4     157      61  52.5      B-
F      S_2      C_1  street_5     159      97  72.2      B+
M      S_2      C_1  street_4     170      81  34.2       A
       S_2      C_2  street_5     193     100  39.1       B
F      S_2      C_2  street_7     194      77  68.5      B+
M      S_2      C_2  street_4     155      91  73.8      A+
       S_2      C_2  street_1     175      74  47.2      B-
F      S_2      C_2  street_7     183      76  85.4       B
       S_2      C_3  street_4     157      78  72.3      B+
M      S_2      C_3  street_5     171      88  32.7       A
F      S_2      C_3  street_7     190      99  65.9       C
       S_2      C_3  street_6     164      81  95.5      A-
M      S_2      C_3  street_4     187      73  48.9       B
F      S_2      C_4  street_2     192      62  45.3       A
M      S_2      C_4  street_7     166      82  48.7       B
F      S_2      C_4  street_6     158      60  59.7      B+
       S_2      C_4  street_2     160      84  67.7       B
       S_2      C_4  street_6     193      54  47.6       B

Class   Address  Height  Weight  Math Physics
Gender School                                              
M      S_1      C_1  street_1     173      63  34.0      A+
F      S_1      C_1  street_2     192      73  32.5      B+
M      S_1      C_1  street_2     186      82  87.2      B+
F      S_1      C_1  street_2     167      81  80.4      B-
       S_1      C_1  street_4     159      64  84.8      B+

### 2. grouby的对象特点
查看所有可调用的方法  
分组对象的head和first  
分组依据  
groupby的[]操作  
连续性变量分组

In [48]:
# a 查看所有可调用的方法
print([attr for attr in dir(group_single) if not attr.startswith('_')])

['Address', 'Class', 'Gender', 'Height', 'Math', 'Physics', 'School', 'Weight', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'var']


In [53]:
# b分组对象的head和first
# 对分组对象使用head函数，返回的是每个组的前几行，而不是数据集前几行
group_single.head(2) 

# first显示的是以分组为索引的每组的第一个分组信息
group_single.first()
group_mul.first()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
2101,S_2,C_1,M,street_7,174,84,83.3,C
2102,S_2,C_1,F,street_6,161,61,50.6,B+


,Class,Gender,Address,Height,Weight,Math,Physics
School,,,,,,,
S_1,C_1,M,street_1,173,63,34.0,A+
S_2,C_1,M,street_7,174,84,83.3,C


Gender   Address  Height  Weight  Math Physics
School Class                                               
S_1    C_1        M  street_1     173      63  34.0      A+
       C_2        M  street_5     188      68  97.0      A-
       C_3        M  street_4     161      68  31.5      B+
S_2    C_1        M  street_7     174      84  83.3       C
       C_2        M  street_5     193     100  39.1       B
       C_3        F  street_4     157      78  72.3      B+
       C_4        F  street_2     192      62  45.3       A

In [55]:
# c 分组依据
# 对于groupby函数而言，分组的依据是非常自由的，只要是与数据框长度相同的列表即可，同时支持函数型分组

df.groupby(np.random.choice(['a','b','c'], df.shape[0])).get_group('a').head()
np.random.choice(['a','b','c'], df.shape[0])
# 相当于将np.random.choice(['a','b','c'],df.shape[0])当做新的一列进行分组

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1201,S_1,C_2,M,street_5,188,68,97.0,A-
1202,S_1,C_2,F,street_4,176,94,63.5,B-


array(['b', 'c', 'a', 'b', 'b', 'b', 'a', 'a', 'b', 'c', 'c', 'c', 'b',
       'b', 'a', 'b', 'b', 'b', 'c', 'b', 'b', 'c', 'b', 'c', 'b', 'c',
       'a', 'c', 'a', 'a', 'a', 'a', 'b', 'a', 'c'], dtype='<U1')

In [68]:
# 从原理上说，我们可以看到利用函数时，传入的对象就是索引，因此根据这一特性可以做一些复杂的操作
df[:5]
df[:5].groupby(lambda x:print(x)).head() # 默认匿名函数x就是索引
df[:5].groupby(lambda x:x).head(2) 
# groupby的head是取每个分组的前几行，这里每个分组只有一行，所以head(1)和head(2)结果一样

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


1101
1102
1103
1104
1105


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [75]:
# 根据奇偶行分组
df.groupby(lambda x:'奇数行' if not df.index.get_loc(x)%2 == 1 else '偶数行').groups

# 如果是多层索引，那么lambda表达式中的输入就是元组，
# 下面实现的功能为查看两所学校中男女生分别均分是否及格
# 注意：此处只是演示groupby的用法，实际操作不会这样写
math_score = df.set_index(['Gender','School'])['Math'].sort_index() # 以Gender和School为索引且排序，然后提取Math列
grouped_score = df.set_index(['Gender','School']).sort_index().\
            groupby(lambda x:(x,'均分及格' if math_score[x].mean() >= 60 else '均分不及格'))
for name, _ in grouped_score:print(name)

{'偶数行': Int64Index([1102, 1104, 1201, 1203, 1205, 1302, 1304, 2101, 2103, 2105, 2202,
             2204, 2301, 2303, 2305, 2402, 2404],
            dtype='int64', name='ID'),
 '奇数行': Int64Index([1101, 1103, 1105, 1202, 1204, 1301, 1303, 1305, 2102, 2104, 2201,
             2203, 2205, 2302, 2304, 2401, 2403, 2405],
            dtype='int64', name='ID')}

(('F', 'S_1'), '均分及格')
(('F', 'S_2'), '均分及格')
(('M', 'S_1'), '均分及格')
(('M', 'S_2'), '均分不及格')


In [77]:
# d groupby的[]操作
# 可以用[]选出groupby对象的某个或者某几个列，上面的均分比较可以如下简洁地写出：
df.groupby(['Gender','School'])['Math'].mean() >= 60

# 还可以直接多个列进行计算统计
df.groupby(['Gender','School'])[['Math','Height']].mean()

Gender  School
F       S_1        True
        S_2        True
M       S_1        True
        S_2       False
Name: Math, dtype: bool

Math      Height
Gender School                       
F      S_1     64.100000  173.125000
       S_2     66.427273  173.727273
M      S_1     63.342857  178.714286
       S_2     51.155556  172.000000

In [78]:
# e 连续型变量分组
# 例如利用cut函数对数学成绩进行分组
bins = [0,40,60,80,90,100]
cuts = pd.cut(df['Math'],bins=bins)
df.groupby(cuts)['Math'].count()

Math
(0, 40]       7
(40, 60]     10
(60, 80]      9
(80, 90]      7
(90, 100]     2
Name: Math, dtype: int64

## 三、聚合、过滤和变换
1. **聚合**   
常用聚合函数    
同时使用多个聚合函数  
使用自定义函数  
利用NameAgg函数  
带参数的聚合函数

2. **过滤 Filteration**  
filter函数是用来筛选某些组的（务必记住结果是组的全体），因此传入的值应当是布尔标量

3. **变换 Transformation**  
传入对象    
利用变换方法进行组内标准化  
利用变换方法进行组内缺失值的均值填充  

### 1. 聚合
a）常用聚合函数  
所谓聚合就是把一堆数，变成一个标量，因此mean/sum/size/count/std/var/sem/describe/first/last/nth/min/max都是聚合函数  
为了熟悉操作，不妨验证标准误sem函数，它的计算公式是：$\frac{组内标准差}{\sqrt{组容量}}$，下面进行验证：

In [82]:
group_m = group_single['Math']
group_m.std().values / np.sqrt(group_m.count().values)
group_m.sem().values

array([5.95857818, 3.93308821])

array([5.95857818, 3.93308821])

In [86]:
# b 同时使用多个聚合函数
group_m.agg(['sum','mean','std']) # 用[] 表示

## 利用元祖进行重命名 
group_m.agg([('Rename_sum','sum'),('Rename_mean','mean')]) # 在[]中用()表示

## 指定哪些函数作用哪些列
group_mul.agg({'Math':['mean','max'], 'Height':'var'}) # 用{}表示，里面是类似字典的结构

,sum,mean,std
School,,,
S_1,956.2,63.746667,23.077474
S_2,1191.1,59.555000,17.589305


,Rename_sum,Rename_mean
School,,
S_1,956.2,63.746667
S_2,1191.1,59.555000


Math       Height
               mean   max    var
School Class                    
S_1    C_1    63.78  87.2  183.3
       C_2    64.30  97.0  132.8
       C_3    63.16  87.7  179.2
S_2    C_1    58.56  83.3   54.7
       C_2    62.80  85.4  256.0
       C_3    63.06  95.5  205.7
       C_4    53.80  67.7  300.2

In [98]:
# c 使用自定义函数
group_single['Math'].agg(lambda x: print(x.head(), '间隔'))
#可以发现，agg函数的传入是分组逐列进行的，有了这个特性就可以做许多事情

## group_single['Math'].agg(lambda x: x.head()) # 报错 Must produce aggregated value
group_single['Math'].agg(lambda x: x.sum())

## 官方并没有提供极差计算函数，但通过agg可以容易地实现组内极差计算
group_single.agg(lambda x: x.max() - x.min())

1101    34.0
1102    32.5
1103    87.2
1104    80.4
1105    84.8
Name: Math, dtype: float64 间隔
2101    83.3
2102    50.6
2103    52.5
2104    72.2
2105    34.2
Name: Math, dtype: float64 间隔


School
S_1    None
S_2    None
Name: Math, dtype: object

School
S_1     956.2
S_2    1191.1
Name: Math, dtype: float64

,Height,Weight,Math
School,,,
S_1,36,41,65.5
S_2,39,46,62.8


In [102]:
# d 利用NamedAgg函数进行多个聚合
## 注意：不支持lambda函数，但是可以使用外置的def函数
def R1(x):
    return x.max() - x.min()
def R2(x):
    return x.max() - x.median() 

group_single['Math'].agg(min_score = pd.NamedAgg(column='clo1', aggfunc=R1),
                         max_score = pd.NamedAgg(column='col2', aggfunc='max'),
                         range_score = pd.NamedAgg(column='col3', aggfunc=R2))

,min_score,max_score,range_score
School,,,
S_1,65.5,97.0,33.5
S_2,62.8,95.5,39.4


In [105]:
# e 带参数的聚合函数
## 判断是否组内数学分数 至少有一个值在50-52之间：
def f(s, low, high):
    return s.between(low,high).max()

group_single['Math'].agg(f,50,52)

## 如果需要使用多个函数，并且其中至少有一个带参数，则使用wrap技巧：
def f_test(s,low,high):
    return s.between(low,high).max()

def agg_f(f_mul,name,*args,**kwargs):
    def wrapper(x):
        return f_mul(x,*args,**kwargs)
    wrapper.__name__ = name
    return wrapper
new_f = agg_f(f_test,'at_least_one_in_50_52',50,52)
group_single['Math'].agg([new_f,'mean']).head()

School
S_1    False
S_2     True
Name: Math, dtype: bool

,at_least_one_in_50_52,mean
School,,
S_1,False,63.746667
S_2,True,59.555000


### 2. 过滤
filter函数是用来筛选某些组的（务必记住结果是组的全体），因此传入的值应当是布尔标量

In [123]:
group_single[['Math','Physics']].filter(lambda x:(x['Math']>32).all()) # all()不能去掉，去掉报错 filter function returned a Series, but expected a scalar bool
group_single[['Math','Physics']].filter(lambda x:(x['Math']>33).all()) # 返回空
# 即要么组全要，要么组全不要？

# all() 函数用于判断给定的可迭代参数 iterable 中的所有元素是否都为 TRUE，如果是返回 True，否则返回 False。
# any() 函数用于判断给定的可迭代参数 iterable 是否全部为 False，则返回 False，如果有一个为 True，则返回 True。

,Math,Physics
ID,,
2101,83.3,C
2102,50.6,B+
2103,52.5,B-
2104,72.2,B+
2105,34.2,A
2201,39.1,B
2202,68.5,B+
2203,73.8,A+
2204,47.2,B-


,Math,Physics
ID,,


### 3. 变换
传入对象  
利用变换方法进行组内标准化  
利用变换方法进行组内缺失值的均值填充  

In [169]:
# a）传入对象
## transform函数中传入的对象是组内的列，并且返回值需要与列长完全一致
group_single[['Math','Height']].transform(lambda x: (x-x.min())).head()

## 如果返回了标量值，那么组内的所有元素会被广播为这个值
group_single[['Math','Height']].transform(lambda x:x.mean()).head()

# b）利用变换方法进行组内标准化
group_single[['Math','Height']].transform(lambda x:(x-x.mean())/x.std()).head()


,Math,Height
ID,,
1101,2.5,14
1102,1.0,33
1103,55.7,27
1104,48.9,8
1105,53.3,0


,Math,Height
ID,,
1101,63.746667,175.733333
1102,63.746667,175.733333
1103,63.746667,175.733333
1104,63.746667,175.733333
1105,63.746667,175.733333


,Math,Height
ID,,
1101,-1.288991,-0.214991
1102,-1.353990,1.279460
1103,1.016287,0.807528
1104,0.721627,-0.686923
1105,0.912289,-1.316166


In [139]:
# c）利用变换方法进行组内缺失值的均值填充
df_nan = df[['Math', 'School']].copy().reset_index()
df_nan.loc[np.random.randint(0,df.shape[0],25), ['Math']] = np.nan
df_nan.head()

df_nan.groupby('School').transform(lambda x: x.fillna(x.mean())).join(df.reset_index()['School']).head()

,ID,Math,School
0,1101,34.0,S_1
1,1102,NaN,S_1
2,1103,87.2,S_1
3,1104,80.4,S_1
4,1105,84.8,S_1


,ID,Math,School
0,1101,34.0,S_1
1,1102,70.4,S_1
2,1103,87.2,S_1
3,1104,80.4,S_1
4,1105,84.8,S_1


## 四、apply函数
1. **apply函数的灵活性**  
标量返回值  
列表返回值  
数据框返回值  
2. **用apply同时统计多个指标**


### 1. apply函数的灵活性
可能在所有的分组函数中，apply是应用最为广泛的，这得益于它的灵活性  
对于传入值而言，是以分组的表传入apply中  
apply函数的灵活性很大程度来源于其返回值的多样性  

In [140]:
df.groupby('School').apply(lambda x:print(x.head(1)))

     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
1101    S_1   C_1      M  street_1     173      63  34.0      A+
     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
2101    S_2   C_1      M  street_7     174      84  83.3       C


""


In [151]:
# 标量返回值
df[['School', 'Math', 'Height']].groupby('School').apply(lambda x:x.max())

# 列表返回值
df[['School','Math','Height']].groupby('School').apply(lambda x: x-x.min()).head()

# 数据框返回值
df[['School', 'Math', 'Height']].groupby('School')\
    .apply(lambda x: pd.DataFrame({'col1': x['Math'] - x['Math'].max(),
                                   'col2': x['Math'] - x['Math'].min(),
                                   'col3': x['Height'] - x['Height'].max(),
                                   'col4': x['Height'] - x['Height'].min()})).head()

,School,Math,Height
School,,,
S_1,S_1,97.0,195
S_2,S_2,95.5,194


,Math,Height
ID,,
1101,2.5,14.0
1102,1.0,33.0
1103,55.7,27.0
1104,48.9,8.0
1105,53.3,0.0


,col1,col2,col3,col4
ID,,,,
1101,-63.0,2.5,-22,14
1102,-64.5,1.0,-3,33
1103,-9.8,55.7,-9,27
1104,-16.6,48.9,-28,8
1105,-12.2,53.3,-36,0


### 2. 用apply同时统计多个指标
此处可以借助OrderedDict工具进行快捷的统计：

In [154]:
# 特征工程中可以使用类似思想构造多变量见的交互统计特征
from collections import OrderedDict
def f(df):
    data = OrderedDict()
    data['M_sum'] = df['Math'].sum()
    data['W_var'] = df['Weight'].var()
    data['H_mean'] = df['Height'].mean()
    return pd.Series(data)
group_single.apply(f)

,M_sum,W_var,H_mean
School,,,
S_1,956.2,117.428571,175.733333
S_2,1191.1,181.081579,172.950000


## 五、问题与练习

### 1. 问题
**【问题一】 什么是fillna的前向/后向填充，如何实现？**

前向填充就是用缺失值的前面一个值填补；  
后向填充就是用缺失值的后面一个值填补；  

有个专门的method方法，可以直接设定，bfill表示用后一个有效行填充，ffill为前一个有效行，nearest是指最近的


**【问题二】 下面的代码实现了什么功能？请仿照设计一个它的groupby版本。**

In [161]:
s = pd.Series ([0, 1, 1, 0, 1, 1, 1, 0])
s1 = s.cumsum()
result = s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1,fill_value =0)

In [162]:
# cumsum 代表计算累加值
# .mul()代表点乘， 即 a * b = c， a是s里的元素，b是s1里的元素,
# diff()代表取差分，默认是一阶差分
s1 # 累计求和
s.mul(s1) # s和s1的点乘
s.mul(s1).diff() # 取差分，默认是一阶差分， 即 新值 = 当前值 - 上一个值 
s.mul(s1).diff().where(lambda x: x < 0) # 判断元素是否<0，注意在这一步 >0 的值被排除变成nan了
s.mul(s1).diff().where(lambda x: x < 0).ffill() # 在筛选的基础上， 空值用前值填充。 
s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1,fill_value =0) # 在上一步结果的基础上，加上s1的值

# 从 [0, 1, 1, 0, 1, 1, 1, 0]
# 到 [0, 1, 2, 0, 1, 2, 3, 0]

0    0
1    1
2    2
3    2
4    3
5    4
6    5
7    5
dtype: int64

0    0
1    1
2    2
3    0
4    3
5    4
6    5
7    0
dtype: int64

0    NaN
1    1.0
2    1.0
3   -2.0
4    3.0
5    1.0
6    1.0
7   -5.0
dtype: float64

0    NaN
1    NaN
2    NaN
3   -2.0
4    NaN
5    NaN
6    NaN
7   -5.0
dtype: float64

0    NaN
1    NaN
2    NaN
3   -2.0
4   -2.0
5   -2.0
6   -2.0
7   -5.0
dtype: float64

0    0.0
1    1.0
2    2.0
3    0.0
4    1.0
5    2.0
6    3.0
7    0.0
dtype: float64

**【问题三】 如何计算组内0.25分位数与0.75分位数？要求显示在同一张表上。**   
先通过变换求和结果，然后再拼接？  
  
**【问题四】 既然索引已经能够选出某些符合条件的子集，那么filter函数的设计有什么意义？**  
因为是对整体操作的，可以节省内存，提高性能  
  
**【问题五】 整合、变换、过滤三者在输入输出和功能上有何异同？**   
整合的输入为 必须指明的聚合函数形式  
变换的输入为 相对整合更自由一点  
过滤的输入为 组的整体，不可以单个处理。  

**【问题六】 在带参数的多函数聚合时，有办法能够绕过wrap技巧实现同样功能吗？**  
不知道....

### 2. 练习
#### 【练习一】： 
现有一份关于diamonds的数据集，列分别记录了克拉数、颜色、开采深度、价格，请解决下列问题：  
(a) 在所有重量超过1克拉的钻石中，价格的极差是多少？   
(b) 若以开采深度的0.2\0.4\0.6\0.8分位数为分组依据，每一组中钻石颜色最多的是哪一种？该种颜色是组内平均而言单位重量最贵的吗？    
(c) 以重量分组(0-0.5,0.5-1,1-1.5,1.5-2,2+)，按递增的深度为索引排序，求每组中连续的严格递增价格序列长度的最大值。   
(d) 请按颜色分组，分别计算价格关于克拉数的回归系数。（单变量的简单线性回归，并只使用Pandas和Numpy完成）    

In [240]:
diamonds = pd.read_csv('./data/Diamonds.csv')
diamonds.head()
diamonds.info()

,carat,color,depth,price
0,0.23,E,61.5,326
1,0.21,E,59.8,326
2,0.23,E,56.9,327
3,0.29,I,62.4,334
4,0.31,J,63.3,335


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   carat   53940 non-null  float64
 1   color   53940 non-null  object 
 2   depth   53940 non-null  float64
 3   price   53940 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 1.6+ MB


In [283]:
# a) 在所有重量超过1克拉的钻石中，价格的极差是多少？
## 极差为17561
## 构造一个值有一样的的组
diamonds[diamonds['carat'] > 1.0].groupby(lambda x:'a').agg(lambda x: x.max() - x.min())

# b) 以开采深度的0.2\0.4\0.6\0.8分位数为分组依据，每一组中钻石颜色最多的是哪一种？该种颜色是组内平均而言单位重量最贵的吗？
## (43.0, 60.8] 是E ， 其他都是G
## max_depth = diamonds['depth'].max()
## bins = [0,max_depth*0.2,max_depth*0.4,max_depth*0.6,max_depth*0.8,max_depth]
## 为啥根据最大值*0.2 不对。。
bins = diamonds['depth'].quantile(np.linspace(0,1,6)).tolist()
cuts = pd.cut(diamonds['depth'],bins=bins)
diamonds['bin_cuts'] = cuts
diamonds.groupby(['bin_cuts'])['color'].describe()

## 最大的两个区间颜色对应是最贵的，前三个不是
diamonds['avg_price'] = diamonds['price'] / diamonds['carat'] # 均重价格
group_avg_price = diamonds.groupby(['bin_cuts','color'])['avg_price'].mean() # 按bins和color分组统计均重价格的均值
group_avg_price.groupby('bin_cuts').idxmax().values


# c) 以重量分组(0-0.5,0.5-1,1-1.5,1.5-2,2+)，按递增的深度为索引排序，求每组中连续的严格递增价格序列长度的最大值。

# d) 请按颜色分组，分别计算价格关于克拉数的回归系数。（单变量的简单线性回归，并只使用Pandas和Numpy完成）






,carat,depth,price,avg_price
a,4.0,35.2,17561,16603.603435


,count,unique,top,freq
bin_cuts,,,,
"(43.0, 60.8]",11294,7,E,2259
"(60.8, 61.6]",11831,7,G,2593
"(61.6, 62.1]",10403,7,G,2247
"(62.1, 62.7]",10137,7,G,2193
"(62.7, 79.0]",10273,7,G,2000


array([(Interval(43.0, 60.8, closed='right'), 'I'),
       (Interval(60.8, 61.6, closed='right'), 'F'),
       (Interval(61.6, 62.1, closed='right'), 'F'),
       (Interval(62.1, 62.7, closed='right'), 'G'),
       (Interval(62.7, 79.0, closed='right'), 'G')], dtype=object)

#### 【练习二】：
有一份关于美国10年至17年的非法药物数据集，列分别记录了年份、州（5个）、县、药物类型、报告数量，请解决下列问题：  
(a) 按照年份统计，哪个县的报告数量最多？这个县所属的州在当年也是报告数最多的吗？  
(b) 从14年到15年，Heroin的数量增加最多的是哪一个州？它在这个州是所有药物中增幅最大的吗？若不是，请找出符合该条件的药物。  

In [309]:
Drug = pd.read_csv('data/Drugs.csv')
Drug.head()
Drug.info()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24062 entries, 0 to 24061
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   YYYY           24062 non-null  int64 
 1   State          24062 non-null  object
 2   COUNTY         24062 non-null  object
 3   SubstanceName  24062 non-null  object
 4   DrugReports    24062 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 940.0+ KB


In [311]:
# a) FRANKLIN县报告数量最多
idx=pd.IndexSlice
for i in range(2010,2018):
    county = (Drug.groupby(['COUNTY','YYYY']).sum().loc[idx[:,i],:].idxmax()[0][0])
    state = Drug.query('COUNTY == "%s"'%county)['State'].iloc[0]
    state_true = Drug.groupby(['State','YYYY']).sum().loc[idx[:,i],:].idxmax()[0][0]
    if state==state_true:
        print('在%d年，%s县的报告数最多，它所属的州%s也是报告数最多的'%(i,county,state))
    else:
        print('在%d年，%s县的报告数最多，但它所属的州%s不是报告数最多的，%s州报告数最多'%(i,county,state,state_true))

在2010年，PHILADELPHIA县的报告数最多，它所属的州PA也是报告数最多的
在2011年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2012年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2013年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2014年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2015年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2016年，HAMILTON县的报告数最多，它所属的州OH也是报告数最多的
在2017年，HAMILTON县的报告数最多，它所属的州OH也是报告数最多的


In [307]:
#(b) 从14年到15年，Heroin的数量增加最多的是哪一个州？它在这个州是所有药物中增幅最大的吗？若不是，请找出符合该条件的药物。
## 数量增加最多，是用极差还是求和?
## 求和是OH， 极差是PA
Drug_filter = Drug[(Drug['YYYY']>=2014) & (Drug['YYYY']<=2015) & (Drug['SubstanceName'] == 'Heroin')]
Drug_filter.groupby('State')['DrugReports'].agg('sum') # agg('lambda x: x.max() - x.min()')
Drug_filter.groupby('State')['DrugReports'].agg(lambda x: x.max() - x.min()) # agg('lambda x: x.max() - x.min()')


Drug_filter = Drug[(Drug['YYYY']>=2014) & (Drug['YYYY']<=2015) & (Drug['State'] == 'OH')]
Drug_filter.groupby('SubstanceName')['DrugReports'].agg('sum').idxmax() #agg(lambda x: x.max() - x.min()).idxmax()

Drug_filter = Drug_filter.groupby(['YYYY','SubstanceName']).sum()
display((Drug_filter.loc[2015]-Drug_filter.loc[2014]).idxmax()) #这里利用了索引对齐的特点
display((Drug_filter.loc[2015]/Drug_filter.loc[2014]).idxmax())

State
KY     8407
OH    43937
PA    37418
VA     6715
WV     2651
Name: DrugReports, dtype: int64

State
KY    1357
OH    4166
PA    4983
VA     555
WV     279
Name: DrugReports, dtype: int64

'Heroin'

DrugReports    Heroin
dtype: object

DrugReports    Acetyl fentanyl
dtype: object